In [9]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

In [10]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [11]:
import torch.nn as nn
import torch.nn.functional as F

class kamnist(nn.Module):
    def __init__(self):
        super(kamnist, self).__init__()
        self.IMAGE_SIZE = 784
        self.FC2_SIZE = 512
        self.OUTPUT_SIZE = 10
        self.fc1 = nn.Linear(self.IMAGE_SIZE, self.FC2_SIZE)
        self.fc2 = nn.Linear(self.FC2_SIZE, self.FC2_SIZE)
        self.fc3 = nn.Linear(self.FC2_SIZE, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, self.OUTPUT_SIZE)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        # flatten x
        x = x.view(-1, self.IMAGE_SIZE)
        x = F.relu(self.fc1(x))

        return x
model = kamnist()
print(model)

kamnist(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [13]:
EPOCHS = 35
model.train()
for EPOCH in range(EPOCHS):
    train_loss = 0.0
    for data, target in train_loader:
         optimizer.zero_grad()
         output = model(data)
         loss = criterion(output, target)
         loss.backward()
         optimizer.step()
         train_loss += loss.item()*data.size(0)

    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        EPOCH+1, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.998775
Epoch: 2 	Training Loss: 0.414624
Epoch: 3 	Training Loss: 0.371115
Epoch: 4 	Training Loss: 0.349557
Epoch: 5 	Training Loss: 0.335909
Epoch: 6 	Training Loss: 0.326222
Epoch: 7 	Training Loss: 0.318866
Epoch: 8 	Training Loss: 0.313022
Epoch: 9 	Training Loss: 0.308227
Epoch: 10 	Training Loss: 0.304197
Epoch: 11 	Training Loss: 0.300742
Epoch: 12 	Training Loss: 0.297736
Epoch: 13 	Training Loss: 0.295087
Epoch: 14 	Training Loss: 0.292726
Epoch: 15 	Training Loss: 0.290604
Epoch: 16 	Training Loss: 0.288682
Epoch: 17 	Training Loss: 0.286929
Epoch: 18 	Training Loss: 0.285320
Epoch: 19 	Training Loss: 0.283837
Epoch: 20 	Training Loss: 0.282463
Epoch: 21 	Training Loss: 0.281184
Epoch: 22 	Training Loss: 0.279990
Epoch: 23 	Training Loss: 0.278871
Epoch: 24 	Training Loss: 0.277819
Epoch: 25 	Training Loss: 0.276828
Epoch: 26 	Training Loss: 0.275892
Epoch: 27 	Training Loss: 0.275005
Epoch: 28 	Training Loss: 0.274163
Epoch: 29 	Training Loss: 0.2

In [14]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.272363

Test Accuracy of     0: 98% (962/980)
Test Accuracy of     1: 97% (1110/1135)
Test Accuracy of     2: 89% (920/1032)
Test Accuracy of     3: 90% (916/1010)
Test Accuracy of     4: 92% (911/982)
Test Accuracy of     5: 87% (783/892)
Test Accuracy of     6: 94% (909/958)
Test Accuracy of     7: 91% (945/1028)
Test Accuracy of     8: 88% (865/974)
Test Accuracy of     9: 90% (917/1009)

Test Accuracy (Overall): 92% (9238/10000)
